In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 33.0 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 101 kB 2.9 MB/s 
     |████████████████████████████████| 6.6 MB 58.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os

# For data manipulation
import numpy as np
import pandas as pd
import warnings
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
from pylab import rcParams

import torch
import torch.nn as nn

# For Transformer Models
from transformers import (
    AdamW,
    AutoTokenizer, 
    pipeline,
    AutoModel,
    AutoConfig,
    AutoModelForSequenceClassification, 
    AutoModelForTokenClassification,
    DataCollatorForLanguageModeling, 
    get_scheduler,
    Trainer, 
    TrainingArguments
)

from tqdm.auto import tqdm

## Загрузка данных

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/skillfactory-hackaton-team9/twitter-analyzer/main/parsed_data/twitter_new_raw_data.csv", index_col=0)

In [7]:
df.head()

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes
0,2022-07-22T20:22:11.000Z,Светлана,@Svetlan99809534,по оценке журнала «Forbes» Керимов стал третьи...,0,0,0
1,2022-07-18T12:17:28.000Z,Серж,@eATOuWB6hfHqVpc,Хаматова в свой фонд подари жизнь по 1млрд в г...,0,0,4
2,2022-07-04T05:47:11.000Z,ленаelena,@elena53166542,У Хаматовой денег море. Она обокрал фонд Подар...,2,0,1
3,2022-07-02T19:22:11.000Z,Dre1,@DremanerDre1,"Мих, ты не ""Лёняверим"", может кто-либо из твои...",0,0,0
4,2022-07-01T18:44:57.000Z,"Wishmaster© Vendetta,",–,Дебиторская задолженность фонда «Подари жизнь»...,0,1,3


In [8]:
df.shape

(2533, 7)

In [49]:
def text2toxicity(text, aggregate=False):
    """ Calculate toxicity of a text (if aggregate=True) or a vector of toxicity aspects (if aggregate=False)"""
    with torch.no_grad():
        inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(model.device)
        proba = torch.sigmoid(model(**inputs).logits).cpu().numpy()
    if isinstance(text, str):
        proba = proba[0]
    if aggregate:
        return 1 - proba.T[0] * (1 - proba.T[-1])
    return proba

## Предсказания первой модели

In [50]:
MODEL_NAME = 'cointegrated/rubert-tiny-toxicity'

In [51]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [52]:
df.text.head(10)

0    по оценке журнала «Forbes» Керимов стал третьи...
1    Хаматова в свой фонд подари жизнь по 1млрд в г...
2    У Хаматовой денег море. Она обокрал фонд Подар...
3    Мих, ты не "Лёняверим", может кто-либо из твои...
4    Дебиторская задолженность фонда «Подари жизнь»...
5    Антон с Димой, которые приехали  в фонд "подар...
6    антон и дима посетили фонд "подари жизнь"..\n\...
7    дима с антоном поддержавшие фонд «подари жизнь...
8    Антон и Дима поддерживающие фонд "подари жизнь...
9    она увидела новость и видео что шазовы приехал...
Name: text, dtype: object

In [53]:
sentiment = df.text.apply(text2toxicity)

In [54]:
sentiment_m1_df = pd.DataFrame.from_records(sentiment, columns=["m1_notoxic", "m1_insult", "m1_obscenity", "m1_threat", "m1_dangerous"])

In [55]:
df.join(sentiment_m1_df)

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes,m1_notoxic,m1_insult,m1_obscenity,m1_threat,m1_dangerous
0,2022-07-22T20:22:11.000Z,Светлана,@Svetlan99809534,по оценке журнала «Forbes» Керимов стал третьи...,0,0,0,0.999206,0.000774,0.000202,0.000292,0.437558
1,2022-07-18T12:17:28.000Z,Серж,@eATOuWB6hfHqVpc,Хаматова в свой фонд подари жизнь по 1млрд в г...,0,0,4,0.969807,0.032301,0.001154,0.000317,0.736651
2,2022-07-04T05:47:11.000Z,ленаelena,@elena53166542,У Хаматовой денег море. Она обокрал фонд Подар...,2,0,1,0.999454,0.000700,0.000134,0.000142,0.263747
3,2022-07-02T19:22:11.000Z,Dre1,@DremanerDre1,"Мих, ты не ""Лёняверим"", может кто-либо из твои...",0,0,0,0.999337,0.000653,0.000186,0.000366,0.466459
4,2022-07-01T18:44:57.000Z,"Wishmaster© Vendetta,",–,Дебиторская задолженность фонда «Подари жизнь»...,0,1,3,0.998452,0.001748,0.000132,0.000115,0.279766
...,...,...,...,...,...,...,...,...,...,...,...,...
2528,2017-05-06T18:48:19.000Z,Ирина Громова,@XxVk6kcJpKasbsc,"Злата Травянко 5 лет. Требуется лечение, Вы мо...",0,0,0,0.999830,0.000194,0.000127,0.000128,0.069039
2529,2017-05-06T16:03:59.000Z,Марина Петрова,@SzdhBqOCuMMmzu8,"Миша Суганов 10 лет. Требуется лечение, Вы мож...",0,0,0,0.999758,0.000268,0.000110,0.000143,0.215942
2530,2017-05-06T15:34:57.000Z,Мари Фирсова,@XijM3aUJrimvMXu,"Аделия Ахметшина 1 год. Требуется лечение, Вы ...",0,0,0,0.999854,0.000166,0.000121,0.000156,0.101451
2531,2017-05-06T15:33:39.000Z,Евгения Попова,@gvkKVdJlFz4BwHM,"Катя Богданова 13 лет. Требуется лечение, Вы м...",0,0,0,0.999681,0.000337,0.000141,0.000157,0.290025


In [56]:
df.shape

(2533, 7)

## Вторая модель

In [57]:
MODEL_NAME = 'SkolkovoInstitute/russian_toxicity_classifier'

In [58]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [59]:
df.text.head(10).apply(text2toxicity)

0     [0.9617549, 0.06992281]
1      [0.9212466, 0.1355247]
2    [0.89812875, 0.15502767]
3     [0.16126138, 0.6975852]
4     [0.8802895, 0.17825463]
5     [0.9519729, 0.07804812]
6    [0.9818751, 0.026242357]
7    [0.12501732, 0.71712625]
8      [0.919049, 0.12858465]
9    [0.94666874, 0.09064583]
Name: text, dtype: object

In [60]:
df.loc[9,"text"]

'она увидела новость и видео что шазовы приехали в фонд подари жизнь который помогает детям в борьбе с раком пересмотрела 3 раза и сидит в соплях наши парни такие хорошие у меня нет слов только слезы которые я глотаю прямо сейчас'

In [61]:
sentiment2 = df.text.apply(text2toxicity)

In [62]:
sentiment_m2_df = pd.DataFrame.from_records(sentiment2, columns=["m2_neutral", "m2_toxic",])

## Третья модель (определение эмоций)

In [63]:
MODEL_NAME = 'cointegrated/rubert-tiny2-cedr-emotion-detection'

In [64]:
labels = ["no_emotion", "joy", "sadness", "surprise", "fear", "anger"]

In [65]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [66]:
df.text.head(10).apply(text2toxicity)

0    [0.99119663, 0.005001627, 0.005758982, 0.00485...
1    [0.014797394, 0.9686751, 0.009435466, 0.018443...
2    [0.9908999, 0.0041388078, 0.006805072, 0.00505...
3    [0.97956204, 0.007433666, 0.002904903, 0.00566...
4    [0.9907698, 0.0046692095, 0.00691416, 0.005395...
5    [0.98604786, 0.010197584, 0.0047300993, 0.0047...
6    [0.9684766, 0.01327445, 0.0036684093, 0.005364...
7    [0.33957234, 0.3033183, 0.007062518, 0.0021266...
8    [0.8579681, 0.19763982, 0.004242808, 0.0031628...
9    [0.0067128865, 0.069356784, 0.45901734, 0.0045...
Name: text, dtype: object

In [67]:
df.loc[2,"text"]

'У Хаматовой денег море. Она обокрал фонд Подари жизнь. Там начались проверки, недостача миллиард, Потому и сбежала. Но надо сказать, что денег нет.'

In [68]:
emotions3 = df.text.apply(text2toxicity)

In [69]:
emotions_m3_df = pd.DataFrame.from_records(emotions3, columns=[f"m3_{label}" for label in labels])

## 4-ая модель 

In [70]:
MODEL_NAME = 'Aniemore/rubert-tiny2-russian-emotion-detection'

In [71]:
labels = ['neutral', 'happiness', 'sadness', 'enthusiasm', 'fear', 'anger', 'disgust']

In [72]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)
if torch.cuda.is_available():
    model.cuda()

In [73]:
emotions4 = df.text.apply(text2toxicity)

In [74]:
emotions_m4_df = pd.DataFrame.from_records(emotions4, columns=[f"m4_{label}" for label in labels])

## Объединим результаты

In [75]:
result_df = (
    df.join(sentiment_m1_df).
      join(sentiment_m2_df).
      join(emotions_m3_df).
      join(emotions_m4_df)
)

In [76]:
result_df.head()

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes,m1_notoxic,m1_insult,m1_obscenity,...,m3_surprise,m3_fear,m3_anger,m4_neutral,m4_happiness,m4_sadness,m4_enthusiasm,m4_fear,m4_anger,m4_disgust
0,2022-07-22T20:22:11.000Z,Светлана,@Svetlan99809534,по оценке журнала «Forbes» Керимов стал третьи...,0,0,0,0.999206,0.000774,0.000202,...,0.004857,0.005459,0.005077,0.987943,0.005911,0.007564,0.005373,0.005333,0.005620,0.003407
1,2022-07-18T12:17:28.000Z,Серж,@eATOuWB6hfHqVpc,Хаматова в свой фонд подари жизнь по 1млрд в г...,0,0,4,0.969807,0.032301,0.001154,...,0.018444,0.004878,0.020253,0.043987,0.348249,0.180201,0.023196,0.002875,0.206812,0.010694
2,2022-07-04T05:47:11.000Z,ленаelena,@elena53166542,У Хаматовой денег море. Она обокрал фонд Подар...,2,0,1,0.999454,0.000700,0.000134,...,0.005050,0.005141,0.005481,0.982843,0.005219,0.011761,0.004467,0.005009,0.005505,0.003115
3,2022-07-02T19:22:11.000Z,Dre1,@DremanerDre1,"Мих, ты не ""Лёняверим"", может кто-либо из твои...",0,0,0,0.999337,0.000653,0.000186,...,0.005663,0.006284,0.011269,0.697096,0.006642,0.005628,0.087352,0.013222,0.145647,0.007568
4,2022-07-01T18:44:57.000Z,"Wishmaster© Vendetta,",–,Дебиторская задолженность фонда «Подари жизнь»...,0,1,3,0.998452,0.001748,0.000132,...,0.005395,0.004534,0.005451,0.985294,0.005397,0.008487,0.005297,0.004396,0.005548,0.003071


In [77]:
result_df.shape

(2533, 27)

In [ ]:
result_df.to_csv("sentiment_from_4_models.csv")

In [ ]:
result_df.sort_values("m4_disgust", ascending=False).text.head().values

array(['пиздец я только сейчас узнала что «подари жизнь» которым столько прикрывали и прикрывают хаматову это даже не фонд хаматовой',
       'С подачи И.В.Безруковой интернет разукрашен бредом обо мне и С.В.Безрукове, и опять же устами той, что всех детей предлагала травить зарином. Весёлая компания идиоток, учитывая, что фонд их Подари жизнь тоже для детей, но строящий хосписы для умирающих, а не больницы для лечения',
       'Фонд «Зекет» запустил акцию «Подари надежду на жизнь» https://kazislam.kz/%d1%84%d0%be%d0%bd%d0%b4-%d0%b7%d0%b5%d0%ba%d0%b5%d1%82-%d0%b7%d0%b0%d0%bf%d1%83%d1%81%d1%82%d0%b8%d0%bb-%d0%b0%d0%ba%d1%86%d0%b8%d1%8e-%d0%bf%d0%be%d0%b4%d0%b0%d1%80%d0%b8-%d0%bd/?lang=ru… через @Қазақстандағы Ислам',
       'круто прикрываться больными? Фондом  с хозяевами мз Великобритании? "Вы плюнете мне в лицо, скажете все гадости, какие только хотели, но переведете деньги в фонд «Подари жизнь»\n\nПодробнее на РБК:\nhttps://rbc.ru/society/23/04/2020/5ea120159a79476a5a32a0de?utm_sour

## Определение объектов в тексте

In [29]:
MODEL_NAME = 'KoichiYasuoka/bert-base-russian-upos'

In [30]:
token_classifier = pipeline(
    "token-classification", model=MODEL_NAME, aggregation_strategy="simple"
)


In [31]:
text = "Я ездил в Барселону"
res = token_classifier(text)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [33]:
[x["word"] for x in res if x["entity_group"] == "PROPN"]

['Барселону']

In [45]:
def find_objects(text):
  res = token_classifier(text)
  objs = [x["word"] for x in res if x["entity_group"] == "PROPN"]
  objs = [x.lower() for x in objs]
  objs = list(set(objs))
  objs = " ".join(objs)
  return objs


In [47]:
df.text.head(10).apply(find_objects)

0    россии forbes керимов фонд здоровье фонд горча...
1                                             хаматова
2                                            хаматовой
3                                спартак лёняверим мих
4                     чулпан эдвард хаматовой чесноков
5                                          димой антон
6                                           антон дима
7                                                     
8                                           антон дима
9                                                     
Name: text, dtype: object

In [48]:
series_ojbects = df.text.apply(find_objects)

## Объединим все в один набор

In [78]:
result_df["m5_objects"] = series_ojbects

In [79]:
result_df.head()

,date,author,twitter_name,text,number_of_reply,number_of_retweets,number_of_likes,m1_notoxic,m1_insult,m1_obscenity,...,m3_fear,m3_anger,m4_neutral,m4_happiness,m4_sadness,m4_enthusiasm,m4_fear,m4_anger,m4_disgust,m5_objects
0,2022-07-22T20:22:11.000Z,Светлана,@Svetlan99809534,по оценке журнала «Forbes» Керимов стал третьи...,0,0,0,0.999206,0.000774,0.000202,...,0.005459,0.005077,0.987943,0.005911,0.007564,0.005373,0.005333,0.005620,0.003407,россии forbes керимов фонд здоровье фонд горча...
1,2022-07-18T12:17:28.000Z,Серж,@eATOuWB6hfHqVpc,Хаматова в свой фонд подари жизнь по 1млрд в г...,0,0,4,0.969807,0.032301,0.001154,...,0.004878,0.020253,0.043987,0.348249,0.180201,0.023196,0.002875,0.206812,0.010694,хаматова
2,2022-07-04T05:47:11.000Z,ленаelena,@elena53166542,У Хаматовой денег море. Она обокрал фонд Подар...,2,0,1,0.999454,0.000700,0.000134,...,0.005141,0.005481,0.982843,0.005219,0.011761,0.004467,0.005009,0.005505,0.003115,хаматовой
3,2022-07-02T19:22:11.000Z,Dre1,@DremanerDre1,"Мих, ты не ""Лёняверим"", может кто-либо из твои...",0,0,0,0.999337,0.000653,0.000186,...,0.006284,0.011269,0.697096,0.006642,0.005628,0.087352,0.013222,0.145647,0.007568,спартак лёняверим мих
4,2022-07-01T18:44:57.000Z,"Wishmaster© Vendetta,",–,Дебиторская задолженность фонда «Подари жизнь»...,0,1,3,0.998452,0.001748,0.000132,...,0.004534,0.005451,0.985294,0.005397,0.008487,0.005297,0.004396,0.005548,0.003071,чулпан эдвард хаматовой чесноков


In [80]:
result_df.shape

(2533, 28)

In [81]:
result_df.to_csv("sentiment_from_5_models.csv")